In [1]:
import time
import dill
import pandas as pd
import numpy as np
import math
from matplotlib import pylab as plt
from sklearn.cluster import KMeans
from ipywidgets import interact

In [2]:
import dill
outL = dill.load(open('outlierCrash.pkd','rb'))

In [16]:
recs = {}
now = time.time()
zz = 0

for item in outL:
    fname = 'OutlierDFs/'+str(item[0])+'_'+str(item[1])+'.pkd'
    out1 = dill.load(open(fname,'rb'))
    d = []
    p = []
    for ii,row in out1.iterrows():
        date1 = pd.Timestamp(row['Start_Time']).time()
        d.append(date1.hour+date1.minute/60.0 + date1.second/60.0/60.0)
        p.append(row['Precipitation(in)'])
        
    for jj,pval in enumerate(p):
        if math.isnan(pval):
            p[jj] = 0.0
        
    X = []

    for kk in range(len(p)):
        X.append([p[kk],d[kk]])
    X = np.array(X)
    
    km = KMeans(n_clusters=5,max_iter=20)
    km.fit(X)
    
    centers = km.cluster_centers_
    
    rec = ''
    
    if sum(centers.T[0] > 0.03) >= 3 and (sum(centers.T[1] <= 6) + sum(centers.T[1] >= 20)) >= 2:
        rec = rec.join('Drainage, Lighting')
    elif sum(centers.T[0] > 0.03) >= 3:
        rec = rec.join('Drainage')
    elif (sum(centers.T[1] <= 6) + sum(centers.T[1] >= 20)) >= 2:
        rec = rec.join('Lighting')
    else:
        rec = rec.join('Volume/Geometry')
        
    recs[item] = rec
    if zz % 100 == 0:
        print(zz)
        print(time.time()-now)
    zz += 1

0
0.05063986778259277
100
3.247415781021118
200
6.817210674285889
300
10.028082370758057
400
13.274163961410522
500
16.686888694763184
600
19.654743671417236
700
22.726106882095337
800
25.64225482940674
900
28.435341835021973
1000
31.204024076461792
1100
34.06181502342224
1200
36.84443426132202
1300
39.44180130958557
1400
42.09255051612854
1500
47.3571093082428
1600
52.51877021789551
1700
57.917389154434204
1800
62.94513821601868
1900
67.52295660972595
2000
72.0899748802185
2100
75.81913304328918
2200
79.60454177856445
2300
83.0481390953064
2400
86.47857522964478
2500
89.6927273273468
2600
93.07575368881226
2700
96.12758088111877
2800
98.98525381088257
2900
101.90393424034119
3000
104.66052198410034
3100
107.60447072982788
3200
110.27382755279541
3300
112.996089220047
3400
115.79145526885986
3500
119.63648533821106
3600
123.86676144599915
3700
127.79783749580383
3800
130.89425349235535
3900
133.62656927108765
4000
136.54886531829834
4100
140.88484454154968
4200
144.3542103767395
4300
1

In [23]:
b = 0
for item in recs:
    if recs[item] != 'Volume/Geometry':
        b += 1

In [24]:
b

4512

In [36]:
from matplotlib import pylab as plt
from ipywidgets import interact
from sklearn.cluster import KMeans

def plot_kmeans(X, n=3, start=None):
    colors = ['#ba2121ff', '#42a5f5ff', '#efa016ff', '#000000ff', '#f231f2', '#6f7c91ff']

    if start is not None:
        n = start.shape[0]
    else:
        start = X[::((X.shape[0]//n)+1),:]
    
    def func(train_steps=0):
        
        if train_steps:
            km = KMeans(n_clusters=n, max_iter=train_steps, n_init=1)#, init=start)
            km.fit(X)
            print(km.cluster_centers_)
            centers = km.cluster_centers_.T
            cols = [colors[label] for label in km.labels_]
        else:
            centers = start.T
            cols = '0.5'
        plt.scatter(*X.T, s=20, c=cols)
        plt.scatter(*centers, c=colors[:len(centers.T)], marker='*', s=150,
                    linewidth=1, edgecolors='k')
        #plt.axis('image')
        plt.title('Precipitation vs Time')
        plt.xlabel('Precipitation (in)')
        plt.ylabel('Time (hrs)')
    
    return func

In [65]:
interact(plot_kmeans(np.array(X), n=5), train_steps=(0, 21, 1));

interactive(children=(IntSlider(value=0, description='train_steps', max=21), Output()), _dom_classes=('widget-…

SyntaxError: can't use starred expression here (<ipython-input-23-d43d7c45e284>, line 1)